In this problem we have two objectives
1. minimize the number of monitoring devices
2. make sure all lines have exactly one monitoring device

We model the problem as follows. 
* We flatten the grid by rows, so that the we will refer to the node at $(i, j)$ as the $k$th node, where $k = i * m + j$
* Take $m*n$ binary variables $q_k$ to represent the presence of a monitoring device at $k$th node.

The number of monitoring devices in a given configuration
$$
\sum_{k=0}^{n*m-1} q_k
$$

If $i$th and $k$th nodes are neighbors (i.e. there is a line in between them), then only one of the nodes need to contain a monitoring device. So we can add to the cost the following term for each line
$$
(q_i + q_k - 1)^2 
$$
So the overall cost is
$$
H=\sum_{k=0}^{n*m-1} q_k + \sum_{lines} (q_i + q_k - 1)^2
$$
Note, that the value of this cost function at its minimum is exactly the number of monitoring devices used.
Taking into account, that 
$$
(q_i + q_k - 1)^2 = q_i^2 + q_k^2 + 1 - 2q_i - 2q_k + 2q_iq_k = 2q_iq_k - q_i - q_k + 1
$$
for the overall cost we get (ignoring the constant $1$)
$$
H = \sum_{k=0}^{n*m-1} q_k + \sum_{lines} (2q_iq_k - q_i - q_k)
$$
Considering the fact, that internal nodes always have 4 lines connecting to them; the edge nodes, which are not at corners always have 3 lines, and the corner nodes have 2 lines connecting to them, we reach the following final expression for the cost
$$
H = 2 \sum_{lines} q_iq_k + \sum_{k=0}^{n*m-1} q_k - \sum_{corner lines} (q_i + q_k)  -\sum_{edge lines} (q_i + q_k) - \sum_{internal lines} (q_i + q_k) \\
 = 2 \sum_{lines} q_iq_k + \sum_{k=0}^{n*m-1} q_k - 2\sum_{corner nodes} q_i  -3 \sum_{edge nodes} q_i - 4\sum_{internal nodes} q_i \\ 
 = 2 \sum_{lines} q_iq_k - \sum_{corner nodes} q_i  -2 \sum_{edge nodes} q_i - 3\sum_{internal nodes} q_i
$$

In [11]:
m = 6
n = 6

In [12]:
Q = {}
for i in range(m-1):
    for j in range(n):
        Q[(i * m + j, (i+1) * n + j)] = 2

for j in range(n-1):
    for i in range(m):
        Q[(i * m + j, i * n + j+1)] = 2

for i in range(m):
    for j in range(n):
        if (i == 0 or i == m-1) and (j == 0 or j == n-1):
            Q[(i * m + j, i * m + j)] = -1
        elif (i == 0 or i == m-1) or (j == 0 or j == n-1):
            Q[(i * m + j, i * m + j)] = -2
        else:
            Q[(i * m + j, i * m + j)] = -3

In [13]:
J = {}
h = {}
for i in range(m-1):
    for j in range(n):
        J[(i * m + j, (i+1) * n + j)] = 2

for j in range(n-1):
    for i in range(m):
        J[(i * m + j, i * n + j+1)] = 2

for i in range(m):
    for j in range(n):
        if (i == 0 or i == m-1) and (j == 0 or j == n-1):
            h[(i * m + j)] = -1
        elif (i == 0 or i == m-1) or (j == 0 or j == n-1):
            h[(i * m + j)] = -2
        else:
            h[(i * m + j)] = -3

In [14]:
import dimod

model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.BINARY)
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=100)

In [5]:
from dwave.system import LeapHybridSampler

sampler = LeapHybridSampler()
response = sampler.sample_qubo(Q)

As we can see below, in the response we have samples corresponding to energy $-42$. Remember, that we have ommitted the constant ones from the cost function, when incorporating the sum over lines. In an $m \times n$ grid we have $m * (n-1) + (m-1) * n$ lines. In the case of $6\times6$ grid this number is $60$, so we have omitted overall a $60$ from the cost function. Adding this to the minimum obtained from sampling we get $-42 + 60 = 18$, which the number of monitoring devices used.

In [15]:
response

SampleSet(rec.array([([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1], -42., 1),
           ([0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0], -42., 1),
           ([0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0], -42., 1),
           ([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1], -42., 1),
           ([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1], -42., 1),
           ([0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0], -42., 1),
           ([0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0], -42., 1),
           ([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,